In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
data_dir = '/DS_203/DATA'


train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    width_shift_range=0.1,      # Small horizontal shifts are okay
    height_shift_range=0.05,    # Small vertical shifts are okay
    # No rotation, zoom, or flip since it’s a spectrogram
    validation_split=0.2        # Split for validation
)

In [ ]:
#training data generator from the given directory followed by data augmentation
train_generator = train_datagen.flow_from_directory(
    directory=data_dir,
    target_size=(128, 128),     # Resize to model input size
    batch_size=32,
    class_mode='categorical',
    subset='training'
)



validation_generator = train_datagen.flow_from_directory(
    directory=data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
#CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128,128, 3)),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes = 6, activation='softmax')  # Softmax for multi-class classification
])

# Step 3: Compile the Model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=20,
    callbacks=[early_stopping]
)